# Simple Test Case for Cupric

The equations can be simplified to

$c_{s,xx} = 0$

$c_{c, xx} = 0$

$-c_{s,x} = c_s \left( 1 - \theta \right)$

$-c_{c,x} = c_c \left( j_0 \left(1 - \theta \right) + j_1 \theta \right) $

$\dot{\theta} = c_s \left( 1 - \theta \right) - \left(j_0 \left(1 - \theta\right) + j_1 \theta \right) \theta c_c$

if we assume that a lot of the parameters 1. We have a single parameter $v$. The solution to this is

$ c_s = \frac{1}{2 - \theta}$

and

$ c_c = \frac{1}{1 + j_0 \left(1 - \theta \right) + j_1 \theta} $

and

$0 = \theta^3 \left[j_0 - j_1\right] + \theta^2 \left[3 j_1 - 4 j_0\right] + \theta \left[ 4 j_0 - j_1 + 1\right] - 1 - j_0$

In [1]:
from main import run, eta_func, calc_eta
from toolz.curried import pipe, do
from scipy import roots
from toolz_ import rcompose
import numpy as np


def get_params():
    return dict(
        diff_sup=1.0,
        sup_inf=1.0,
        delta=1.0,
        k_plus=1.0,
        k_minus=1.0,
        gamma=1.0,
        dt=1e+10,
        nx=1000,
        max_steps=1,
        max_sweeps=15,
        v0=1.0,
        vm=0.0,
        output=False,
        alpha=0.4,
        F=96485.332,
        R=8.314,
        T=270.0,
        theta_ini=0.0,
        sup_ini=0.0,
        tf=1.
    )

    
def jj(params):
    return 1. / eta_func(params, params['v0']) 


def update_params(j0, j1):
    return pipe(
        get_params(),
        lambda x: dict(x, j0=jj(x) * j0, j1=jj(x) * j1)
    )


def answer(j0, j1):
    return pipe(
        min(roots([
            j0 - j1,
            2 * j1 - 3 * j0,
            2 * j0 + 1,
            -1
        ])),
        lambda x: (1 / (2 - x), x)
    )


def predict(j0, j1):
    return pipe(
        update_params(j0, j1),
        run,
        lambda x: (x['sup'][0], x['theta']['new'])
    )


def check(j0, j1):
    return np.allclose(answer(j0, j1), predict(j0, j1), rtol=1e-3)


assert check(1, 1)
assert check(10, 1)
assert check(100, 1)